1️⃣ Secrets & API-key bootstrap

In [ ]:
# ────────────────────────────────────────────────────────────────────────
#  SECRETS → LOCAL FILES  |  runs before any other import
# ────────────────────────────────────────────────────────────────────────
import os, base64, pathlib, json
import openai
from openai import OpenAI

def _decode_and_validate(secret_name: str, out_file: str):
    """Decode Base64 GitHub secret and validate JSON structure."""
    try:
        decoded = base64.b64decode(os.environ[secret_name]).decode('utf-8')
        data = json.loads(decoded)

        # For token.json, verify required fields
        if out_file == "token.json":
            required_fields = {'token', 'refresh_token', 'scopes'}
            if not required_fields.issubset(data.keys()):
                missing = required_fields - set(data.keys())
                raise ValueError(f"Missing required fields in token: {missing}")

        pathlib.Path(out_file).write_text(decoded)
        return True
    except Exception as e:
        print(f"❌ Error processing {secret_name}: {str(e)}")
        raise

try:
    _decode_and_validate("CLIENT_SECRETS_JSON", "client_secrets.json")
    _decode_and_validate("TOKEN_JSON", "token.json")
except Exception as e:
    print("❌ Failed to initialize secrets")
    raise

openai.api_key = os.environ["OPENAI_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]  # <-- add this line
client = OpenAI()
print("✅ Secrets decoded and validated, OpenAI client ready.")


2️⃣ Parameter cell

In [ ]:
# PARAMETERS  (Papermill / run-notebook overwrites these at run-time)
GAMEPLAY_URL = None                         # GitHub secret injects real link
TITLE_TEXT   = "Daily DALLE Short"          # you can override this too
OPENAI_MODEL = "gpt-4o"


🔐 3. Authenticate with YouTube

In [ ]:
# 🔐 2. Authenticate with YouTube (automatic – no browser needed)
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json

# Use only the essential scope needed for uploads
SCOPE = "https://www.googleapis.com/auth/youtube.upload"

def get_authenticated_service():
    """Create authenticated YouTube client with scope validation."""
    try:
        # Load token data
        with open("token.json") as f:
            token_data = json.load(f)

        # Verify the token has our required scope
        if 'scopes' not in token_data or SCOPE not in token_data['scopes']:
            raise ValueError(f"Token missing required scope: {SCOPE}")

        creds = Credentials.from_authorized_user_info(token_data, [SCOPE])

        # Refresh token if needed
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                # Update token file with refreshed credentials
                with open("token.json", "w") as f:
                    json.dump(json.loads(creds.to_json()), f)
            except Exception as refresh_error:
                print(f"⚠️ Token refresh failed: {refresh_error}")
                # Continue with expired token if we have one
                if not creds.token:
                    raise

        return build("youtube", "v3", credentials=creds)
    except HttpError as e:
        print(f"❌ YouTube API error: {e}")
        raise
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        raise

try:
    youtube = get_authenticated_service()
    print(f"✅ YouTube API authenticated with scope: {SCOPE}")
except Exception as e:
    print("❌ Failed to initialize YouTube client")
    raise

4-7: THIS IS WHERE WE BUILD OUR SHORTS DRAMA STORIES GENERATION CODE BLOCKS

# BLOCK 1: QUOTES AND CHARACTERS LISTS

In [ ]:
# BLOCK 1: QUOTES AND CHARACTERS LISTS

import random

# A long list of popular, meaningful, or motivational quotes (add more as you wish!)
QUOTES_LIST = [
    "The only way to do great work is to love what you do.",
    "Fall seven times, stand up eight.",
    "The journey of a thousand miles begins with one step.",
    "Success is not final, failure is not fatal: It is the courage to continue that counts.",
    "You miss 100% of the shots you don’t take.",
    "Believe you can and you’re halfway there.",
    "Don’t watch the clock; do what it does. Keep going.",
    "It does not matter how slowly you go as long as you do not stop.",
    "Tough times never last, but tough people do.",
    "Dream big and dare to fail.",
    "Our greatest glory is not in never falling, but in rising every time we fall.",
    "Your life does not get better by chance, it gets better by change.",
    "Act as if what you do makes a difference. It does.",
    "Happiness is not something ready made. It comes from your own actions.",
    "When you arise in the morning, think of what a precious privilege it is to be alive.",
    "Age is an issue of mind over matter. If you don't mind, it doesn't matter.",
    "It's not the years in your life that count. It's the life in your years.",
    "Strength does not come from physical capacity. It comes from an indomitable will.",
    "To love and be loved is to feel the sun from both sides.",
    "The best way to get started is to quit talking and begin doing.",
    "Never let the fear of striking out keep you from playing the game.",
    "Difficulties in life are intended to make us better, not bitter.",
    "If you’re going through hell, keep going.",
    "It always seems impossible until it’s done.",
    "Grow through what you go through.",
    "You must be the change you wish to see in the world.",
    "The harder you work for something, the greater you’ll feel when you achieve it.",
    "Let your smile change the world, but don’t let the world change your smile.",
    "You don’t have to be great to start, but you have to start to be great.",
    "Be yourself; everyone else is already taken.",
    # Add more as needed!
]

# A long, diverse list of characters (feel free to add or change)
CHARACTERS_LIST = [
    "Yoda (Star Wars)",
    "Pikachu (Pokémon)",
    "Tony Soprano (The Sopranos, mafia boss)",
    "Optimus Prime (Transformers)",
    "Gandalf (Lord of the Rings)",
    "SpongeBob SquarePants",
    "Walter White (Breaking Bad)",
    "Darth Vader",
    "Sailor Moon",
    "Bart Simpson",
    "Iron Man",
    "Dwayne 'The Rock' Johnson",
    "Mickey Mouse",
    "Mario (Super Mario Bros)",
    "Batman",
    "Shrek",
    "Homer Simpson",
    "Elsa (Frozen)",
    "Rick Sanchez (Rick and Morty)",
    "Naruto Uzumaki",
    "Michael Scott (The Office)",
    "Freddy Mercury",
    "Vito Corleone (The Godfather, mafia boss)",
    "Goku (Dragon Ball)",
    "Scarlett O’Hara (Gone with the Wind)",
    "Jack Sparrow",
    "Hermione Granger (Harry Potter)",
    "Mr. Bean",
    "Bugs Bunny",
    "Deadpool",
    "Thanos",
    # Feel free to expand with more fun or niche characters!
]

# Function to select random quote and character
def pick_random_quote_and_character():
    quote = random.choice(QUOTES_LIST)
    character = random.choice(CHARACTERS_LIST)
    return quote, character

selected_quote, selected_character = pick_random_quote_and_character()
print(f"Quote: {selected_quote}\nCharacter: {selected_character}")


# BLOCK 2: GENERATE PERSONALIZED SCRIPT WITH GPT API

In [ ]:
def format_character_name(character):
    return character.split('(')[0].strip()

def generate_personalized_script(quote, character):
    character_short = format_character_name(character)
    system_prompt = (
        f"You are {character}, a famous character from media. You speak in their typical voice and style, "
        "adapting advice and quotes to sound just like them. Your audience loves both wisdom and a bit of fun!"
    )
    user_prompt = (
        f'Intro: "Hi there... Are you doing okay? Let {character_short} give you some advice."\n\n'
        f'Please rephrase or remix the following motivational quote so it fits the personality, mood, '
        f'speech pattern, and sense of humor (if any) of {character}. Use references, phrases, or quirks that fans will recognize.\n\n'
        f'Quote: "{quote}"\n\n'
        f'After the quote, add an outro line:\n'
        f'"That\'s right, you hear that!? Now go out there and make a wonderful day! '
        f'See you next time when I say: "\n\n'
        f'Keep the total script short and natural (10 seconds when spoken).'
    )
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=200,
        temperature=0.9,
    )
    script = response.choices[0].message.content
    return script

script = generate_personalized_script(selected_quote, selected_character)
print(script)


3️⃣ Generate Relevant Images with DALL·E 3 (Segmented for Story)

In [ ]:
def generate_dalle_image(prompt, filename):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    image_url = response.data[0].url
    import requests
    from PIL import Image
    from io import BytesIO
    img_data = requests.get(image_url).content
    img = Image.open(BytesIO(img_data))
    img.save(filename)
    return filename

def make_image_prompts(character, quote):
    character_short = format_character_name(character)
    prompt1 = (f"Highly detailed portrait of {character_short}, in a visually stunning, cool, and uplifting setting, "
               f"expressing the character's signature personality and mood. Cinematic, inspiring, sharp focus.")
    prompt2 = (f"{character_short} explaining an important life lesson in their unique style, in a scene that visually fits the quote: '{quote}'. "
               "The character is expressive, perhaps gesturing or in a teaching pose, with a motivational and positive atmosphere. Cinematic, high quality.")
    return [prompt1, prompt2]

def generate_images_for_script(character, quote):
    prompts = make_image_prompts(character, quote)
    image_files = []
    for i, prompt in enumerate(prompts):
        filename = f"character_scene_{i+1}.png"
        generate_dalle_image(prompt, filename)
        image_files.append(filename)
    return image_files

IMAGE_FILES = generate_images_for_script(selected_character, selected_quote)
print("Generated images:", IMAGE_FILES)


4️⃣ Generate Narration with OpenAI TTS (Echo Voice)

In [ ]:
def generate_tts_narration(script, output_path="narration.mp3"):
    voices = ["fable", "onyx", "echo"]
    selected_voice = random.choice(voices)
    print(f"🎤 Selected TTS voice: {selected_voice}")
    tts_response = client.audio.speech.create(
        model="tts-1",
        voice=selected_voice,
        input=script,
        response_format="mp3"
    )
    tts_response.stream_to_file(output_path)
    print(f"Narration audio saved to {output_path}")
    return output_path

NARRATION_AUDIO_PATH = generate_tts_narration(script)


5️⃣ Generate Synced Subtitles (GPT for Chunks Matching Narration Timing)

In [ ]:
from aeneas.executetask import ExecuteTask
from aeneas.task import Task
import tempfile
import os

def split_text_into_max_lines(text, max_width_chars=25, max_lines=4):
    import textwrap
    wrapper = textwrap.TextWrapper(
        width=max_width_chars,
        break_long_words=True,
        break_on_hyphens=True
    )
    lines = wrapper.wrap(text)
    chunks = []
    for i in range(0, len(lines), max_lines):
        chunk = "\n".join(lines[i:i + max_lines])
        chunks.append(chunk)
    return chunks

def chunk_script_for_subtitles(script, narration_audio_path, max_width_chars=25, max_lines=4):
    import json
    with tempfile.NamedTemporaryFile(mode='w+', delete=False, suffix='.txt') as tf:
        tf.write(script)
        transcript_path = tf.name
    config_string = u"task_language=eng|is_text_type=plain|os_task_file_format=json"
    task = Task(config_string=config_string)
    task.audio_file_path_absolute = narration_audio_path
    task.text_file_path_absolute = transcript_path
    task.sync_map_file_path_absolute = transcript_path + ".json"
    ExecuteTask(task).execute()
    task.output_sync_map_file()
    with open(task.sync_map_file_path_absolute, 'r') as f:
        sync_map = json.load(f)
    subtitle_timings = []
    for fragment in sync_map["fragments"]:
        if not fragment["lines"]: continue
        text = fragment["lines"][0].strip()
        start = float(fragment["begin"])
        end = float(fragment["end"])
        duration = end - start
        chunks = split_text_into_max_lines(text, max_width_chars=max_width_chars, max_lines=max_lines)
        chunks = [chunk for chunk in chunks if chunk.strip()]
        if not chunks:
            continue
        elif len(chunks) == 1:
            subtitle_timings.append({"text": chunks[0], "start": start, "end": end})
        else:
            chunk_duration = duration / len(chunks)
            for i, chunk in enumerate(chunks):
                chunk_start = start + i * chunk_duration
                chunk_end = chunk_start + chunk_duration
                subtitle_timings.append({"text": chunk, "start": chunk_start, "end": chunk_end})
    os.remove(transcript_path)
    os.remove(task.sync_map_file_path_absolute)
    return subtitle_timings

SUBTITLES = chunk_script_for_subtitles(script, NARRATION_AUDIO_PATH)


6️⃣ Combine All Into Shorts Video (Images + Narration + Subtitles)

In [ ]:
# BLOCK 6: COMBINE EVERYTHING INTO SHORTS VIDEO WITH PERFECT SUBTITLES

import sys

if sys.platform.startswith("linux"):
    TEXT_FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
elif sys.platform.startswith("win"):
    TEXT_FONT_PATH = "C:/Windows/Fonts/DejaVuSans-Bold.ttf"
else:
    TEXT_FONT_PATH = "DejaVuSans-Bold.ttf"
TEXT_FONTSIZE = 52

def _set_duration(clip, duration):
    if hasattr(clip, "with_duration"): return clip.with_duration(duration)
    return clip.set_duration(duration)

def _set_audio(clip, audio):
    if hasattr(clip, "with_audio"): return clip.with_audio(audio)
    return clip.set_audio(audio)

def _set_position(clip, pos):
    if hasattr(clip, "with_position"): return clip.with_position(pos)
    return clip.set_position(pos)

def _set_start(clip, start):
    if hasattr(clip, "with_start"): return clip.with_start(start)
    return clip.set_start(start)

def resize_fx(clip, newsize):
    if hasattr(clip, "resized"):
        return clip.resized(newsize)
    return clip.resize(newsize)

from moviepy.editor import ImageClip, TextClip, CompositeVideoClip, concatenate_videoclips, AudioFileClip
import textwrap

def wrap_text_for_width(text, max_width_chars=25, max_lines=4):
    """Wrap text to fit within video frame width and limit to max_lines."""
    wrapper = textwrap.TextWrapper(
        width=max_width_chars,
        break_long_words=True,
        break_on_hyphens=True,
        max_lines=max_lines,
        placeholder=' [...]'
    )
    return "\n".join(wrapper.wrap(text))

def robust_textclip(text, start, end, font_path, fontsize):
    """
    Create a TextClip for subtitles, ensuring consistent formatting.
    """
    duration = end - start
    txt = None

    # Wrap text to ensure it fits within frame and limits to max lines
    wrapped_text = wrap_text_for_width(text, max_width_chars=25, max_lines=4)

    # Try caption mode with proper size constraints
    try:
        txt = TextClip(
            wrapped_text,
            font_size=fontsize,
            color='white',
            method='caption',
            size=(900, None)  # Max width for 1080px frame with margins
        )
        return txt
    except Exception as e:
        print(f"[Subtitle/caption failover] {e}")

    # Try label mode with font, fontsize
    try:
        txt = TextClip(
            wrapped_text,
            fontsize=fontsize,
            color='white',
            font=font_path,
            method='label'
        )
        return txt
    except Exception as e:
        print(f"[Subtitle/label failover] {e}")

    # Try plain text, no font specified
    try:
        txt = TextClip(wrapped_text)
        return txt
    except Exception as e:
        print(f"[Subtitle/plain failover] {e}")

    # Last resort: Render text with Pillow, use ImageClip
    try:
        from PIL import Image, ImageDraw, ImageFont
        import numpy as np

        W, H = 900, 200  # Subtitle box (width, height)
        bg_color = (0, 0, 0, 90)  # Semi-transparent background
        fg_color = (255, 255, 255, 255)

        # Load font, fallback to default PIL font if error
        try:
            font = ImageFont.truetype(font_path, fontsize)
        except Exception as e:
            print(f"[Subtitle/Pillow] Can't load '{font_path}', using default font: {e}")
            font = ImageFont.load_default()

        # Use the pre-wrapped text
        final_text = wrapped_text

        # Dummy image for measuring
        dummy_img = Image.new("RGBA", (W, H))
        draw = ImageDraw.Draw(dummy_img)
        bbox = draw.textbbox((0, 0), final_text, font=font)
        text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]

        # Ensure text fits within our box width
        if text_width > W - 40:  # 40px total margin
            # If still too wide, try smaller font or more aggressive wrapping
            scale_factor = (W - 40) / text_width
            adjusted_fontsize = int(fontsize * scale_factor)
            try:
                font = ImageFont.truetype(font_path, adjusted_fontsize)
            except:
                font = ImageFont.load_default()

        # Now render actual image with proper dimensions
        margin_px = 60
        img_height = max(H, text_height + 40 + margin_px)
        img = Image.new('RGBA', (W, img_height), bg_color)
        draw = ImageDraw.Draw(img)

        # Center the text horizontally and vertically within the box
        text_x = (W - text_width) // 2
        text_y = (img_height - text_height - margin_px) // 2

        draw.text(
            (text_x, text_y),
            final_text,
            font=font,
            fill=fg_color,
            align='center'
        )

        np_img = np.array(img)
        txt = ImageClip(np_img, duration=duration)
        return txt
    except Exception as e:
        print(f"[Subtitle/Pillow ultimate fail] {e}")
        raise RuntimeError("Subtitle TextClip/ImageClip creation failed with all methods.")

def create_shorts_video(IMAGE_FILES, NARRATION_AUDIO_PATH, SUBTITLES, output_path="shorts_final.mp4"):
    """
    Creates a 1080x1920 shorts video using your DALL·E images, TTS narration,
    and perfectly synced subtitle overlays (top bar). Ready for YouTube Shorts.
    """
    audio = AudioFileClip(NARRATION_AUDIO_PATH)
    total_duration = audio.duration
    num_images = len(IMAGE_FILES)
    img_duration = total_duration / num_images

    # Prepare image clips (robust resize for 1080x1920)
    clips = []
    for idx, img in enumerate(IMAGE_FILES):
        base_clip = ImageClip(img)
        base_clip = resize_fx(base_clip, (1080, 1920))
        base_clip = _set_duration(base_clip, img_duration)
        base_clip = _set_position(base_clip, 'center')
        clips.append(base_clip)
    video = concatenate_videoclips(clips, method="compose")
    video = _set_audio(video, audio)

    # Prepare subtitle clips (positioned at top with small margin)
    subtitle_clips = []
    for sub in SUBTITLES:
        txt = robust_textclip(
            sub["text"],
            sub["start"],
            sub["end"],
            TEXT_FONT_PATH,
            TEXT_FONTSIZE
        )
        txt = _set_start(txt, sub["start"])
        txt = _set_duration(txt, sub["end"] - sub["start"])
        # Top bar position: 120px from top, centered
        txt = _set_position(txt, ("center", 120))
        # Only call margin if TextClip (not ImageClip)
        if isinstance(txt, TextClip):
            txt = txt.margin(top=10, opacity=0)
        subtitle_clips.append(txt)

    # Overlay subtitles on video
    final = CompositeVideoClip([video] + subtitle_clips)
    final.write_videofile(output_path, fps=24, codec="libx264", audio_codec="aac")
    print(f"Final shorts video saved to {output_path}")
    return output_path

SHORTS_VIDEO_PATH = create_shorts_video(IMAGE_FILES, NARRATION_AUDIO_PATH, SUBTITLES)


#@📝 8. Generate Metadata

In [ ]:
def parse_metadata_response(text):
    title, desc, tags = None, None, []
    for line in text.splitlines():
        low = line.lower()
        if low.startswith("title:"):
            title = line.split(":",1)[1].strip().strip(' "\'')
            if title and not title[-1] in ('!', '?', '.'):
                title += '!'
        if low.startswith("description:"):
            desc = line.split(":",1)[1].strip()
        if low.startswith("tags:"):
            tags = [t.strip() for t in line.split(":",1)[1].split(",")]
    required_tags = ["#shorts", "#short","#Shortsviral", "fyp", "viralshorts", "viralnews", "globalnews", "#trending", "#youtubeshorts"]
    tags = [tag for tag in tags if tag]
    for req_tag in required_tags:
        if req_tag.lower() not in [t.lower() for t in tags]:
            tags.append(req_tag)
    tags = [tag[:25] for tag in tags if tag]
    tags = list(set(tags))[:30]
    return title, desc, tags

def generate_metadata_from_story(story_text):
    prompt = (
        f"You are an expert at viral YouTube Shorts. Given the following short story, create:\n"
        "1. A clickworthy, emotional TITLE (<85 chars) based on the most powerful moment, surprise, or twist (NO bland summaries, NO boring names, must create curiosity!) and include 2-3 relevant hashtags\n"
        "2. A DESCRIPTION (<3500 characters) of the story, opening with a main keyword, includes a call to action (Like/Comment/Share), ends with a question. Then add 20-30 relevant hashtags at the end.\n"
        "3. Around 30 SEO TAGS (comma-separated), mixing story keywords, emotional triggers, setting, and all required viral tags (#shorts, #youtubeshorts, etc).\n"
        "\nStory:\n"
        f"{story_text}\n"
        "\nFormat:\n"
        "Title: ...\nDescription: ...\nTags: ...\n"
    )
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0.8,
        max_tokens=3000
    )
    text = resp.choices[0].message.content.strip()
    return parse_metadata_response(text)

title, description, tags = generate_metadata_from_story(script)
print("📝 Metadata:\n ", title, "\n ", description, "\n ", tags)


📤 9. Upload to YouTube

In [ ]:
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
import time, os

def upload_video_to_youtube(
    file_path, title, description, tags,
    category_id=22, privacy="public",
    chunk_size=1024*1024, max_retries=5
):
    if isinstance(tags, str):
        tags = [t.strip() for t in tags.split(",") if t.strip()]
    if not any(t.lower() == "#shorts" for t in tags):
        tags.append("#shorts")
    body = {
        "snippet": {
            "title": title,
            "description": f"{description}\n\n#shorts",
            "tags": tags,
            "categoryId": str(category_id)
        },
        "status": {
            "privacyStatus": privacy,
            "selfDeclaredMadeForKids": False
        }
    }
    media = MediaFileUpload(
        file_path,
        mimetype='video/mp4',
        chunksize=chunk_size,
        resumable=True
    )
    req = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=media
    )
    done = False
    retry = 0
    while not done:
        try:
            status, resp = req.next_chunk()
            if status:
                print(f"🟢 {int(status.progress()*100)}% uploaded")
            else:
                done = True
        except HttpError as e:
            if e.resp.status in [500,502,503,504] and retry < max_retries:
                retry += 1
                time.sleep(2**retry)
                print(f"⚠️ Retry {retry}")
            else:
                raise
    print("✅ Upload complete! Video ID:", resp['id'])
    return resp

response = upload_video_to_youtube(SHORTS_VIDEO_PATH, title, description, tags)
print("🎉 Done! https://youtu.be/" + response["id"])
